In [10]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('../src/data/data_source.csv')

In [3]:
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


Step 1: Load and Inspect the Data


In [4]:
print(df.info())
print(df.describe())

# Check for missing values
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

Step 2: Handle Missing Values


In [8]:
# Replace missing values for numerical columns with the mean
numerical_columns = ['TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)']
for col in numerical_columns:
    df[col] = df[col].fillna(df[col].mean())

# Replace missing values for categorical columns with the mode
df['Handset Type'] = df['Handset Type'].fillna(df['Handset Type'].mode()[0])


Step 3: Handle Outliers


In [11]:

def handle_outliers(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with the mean
    mean_value = df[column].mean()
    df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound), mean_value, df[column])

# Handle outliers for numerical columns
for col in numerical_columns:
    handle_outliers(col)


Step 4: Aggregate Metrics per Customer


In [13]:
# Group by customer (MSISDN) and aggregate
aggregated_df = df.groupby('MSISDN/Number').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode()[0]  # Most common handset type
}).reset_index()

# Rename columns for clarity
aggregated_df.rename(columns={
    'TCP DL Retrans. Vol (Bytes)': 'Avg_TCP_Retransmission',
    'Avg RTT DL (ms)': 'Avg_RTT',
    'Avg Bearer TP DL (kbps)': 'Avg_Throughput',
    'Handset Type': 'Most_Common_Handset'
}, inplace=True)

# Display the result
print(aggregated_df.head())


   MSISDN/Number  Avg_TCP_Retransmission     Avg_RTT  Avg_Throughput  \
0   3.360100e+10            2.080991e+07   46.000000            37.0   
1   3.360100e+10            2.080991e+07   30.000000            48.0   
2   3.360100e+10            2.080991e+07  109.795706            48.0   
3   3.360101e+10            1.066000e+03   69.000000           204.0   
4   3.360101e+10            1.507977e+07   57.000000         20197.5   

              Most_Common_Handset  
0  Huawei P20 Lite Huawei Nova 3E  
1          Apple iPhone 7 (A1778)  
2                       undefined  
3         Apple iPhone 5S (A1457)  
4         Apple iPhone Se (A1723)  


Step 5: Save the Results


In [14]:
# Save the aggregated data to a CSV file
aggregated_df.to_csv('../src/data/aggregated_user_experience.csv', index=False)


Compute Top, Bottom, and Most Frequent Values

In [16]:
# Function to compute top, bottom, and most frequent values
def get_top_bottom_frequent(df, column):
    top_10 = df[column].nlargest(10)  # Top 10 largest values
    bottom_10 = df[column].nsmallest(10)  # Bottom 10 smallest values
    most_frequent = df[column].mode()  # Most frequent values
    
    return top_10, bottom_10, most_frequent

# Compute for TCP Retransmission
top_tcp, bottom_tcp, frequent_tcp = get_top_bottom_frequent(aggregated_df, 'Avg_TCP_Retransmission')

# Compute for RTT
top_rtt, bottom_rtt, frequent_rtt = get_top_bottom_frequent(aggregated_df, 'Avg_RTT')

# Compute for Throughput
top_throughput, bottom_throughput, frequent_throughput = get_top_bottom_frequent(aggregated_df, 'Avg_Throughput')

# Display results
print("Top 10 TCP Retransmission Values:")
print(top_tcp)
print("\nBottom 10 TCP Retransmission Values:")
print(bottom_tcp)
print("\nMost Frequent TCP Retransmission Values:")
print(frequent_tcp)

print("\nTop 10 RTT Values:")
print(top_rtt)
print("\nBottom 10 RTT Values:")
print(bottom_rtt)
print("\nMost Frequent RTT Values:")
print(frequent_rtt)

print("\nTop 10 Throughput Values:")
print(top_throughput)
print("\nBottom 10 Throughput Values:")
print(bottom_throughput)
print("\nMost Frequent Throughput Values:")
print(frequent_throughput)


Top 10 TCP Retransmission Values:
72271    49986038.0
56192    49969605.0
32676    49777538.0
64581    49619503.0
69194    49599533.0
26714    49591949.0
43785    49408068.0
27234    49398764.0
94060    49384238.0
54920    49279434.0
Name: Avg_TCP_Retransmission, dtype: float64

Bottom 10 TCP Retransmission Values:
56622     2.0
95322     2.0
71705     4.0
71778     4.0
87156     4.0
87348     4.0
95213     4.0
95331     4.0
95774     4.0
100616    4.0
Name: Avg_TCP_Retransmission, dtype: float64

Most Frequent TCP Retransmission Values:
0    2.080991e+07
Name: Avg_TCP_Retransmission, dtype: float64

Top 10 RTT Values:
8031     221.0
25018    221.0
29166    221.0
32506    221.0
37323    221.0
39854    221.0
40029    221.0
46187    221.0
48004    221.0
48314    221.0
Name: Avg_RTT, dtype: float64

Bottom 10 RTT Values:
28878     0.0
40873     0.0
44748     0.0
66098     0.0
88903     0.0
98541     0.0
39066     2.0
83396     4.0
36039     5.0
106569    5.0
Name: Avg_RTT, dtype: float64


Calculate Average Throughput and Average TCP Retransmission per Handset Type

In [18]:
# Group by handset type and calculate the average throughput and TCP retransmission
avg_throughput_per_handset = aggregated_df.groupby('Most_Common_Handset')['Avg_Throughput'].mean().reset_index()
avg_tcp_per_handset = aggregated_df.groupby('Most_Common_Handset')['Avg_TCP_Retransmission'].mean().reset_index()

# Display the results
print("Average Throughput per Handset Type:")
print(avg_throughput_per_handset)

print("\nAverage TCP Retransmission per Handset Type:")
print(avg_tcp_per_handset)


Average Throughput per Handset Type:
                                    Most_Common_Handset  Avg_Throughput
0                            A-Link Telecom I. Cubot A5    23184.000000
1                     A-Link Telecom I. Cubot Note Plus     6527.000000
2                        A-Link Telecom I. Cubot Note S     8515.000000
3                          A-Link Telecom I. Cubot Nova    13300.045927
4                         A-Link Telecom I. Cubot Power    13300.045927
...                                                 ...             ...
1389  Zte Zte Blade C2 Smartphone Android By Sfr Sta...       44.500000
1390                          Zyxel Communicat. Lte7460    39741.000000
1391                          Zyxel Communicat. Sbg3600    13300.045927
1392                    Zyxel Communicat. Zyxel Wah7706     1263.000000
1393                                          undefined     4501.150557

[1394 rows x 2 columns]

Average TCP Retransmission per Handset Type:
                            

K-means Clustering on Experience Metrics (k=3)

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Select relevant columns for clustering
experience_metrics = aggregated_df[['Avg_TCP_Retransmission', 'Avg_RTT', 'Avg_Throughput']]

# Fill missing values with the mean for each column
experience_metrics.fillna(experience_metrics.mean(), inplace=True)

# Normalize the data
scaler = StandardScaler()
normalized_metrics = scaler.fit_transform(experience_metrics)

# Perform K-means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42)
aggregated_df['Experience Cluster'] = kmeans.fit_predict(normalized_metrics)

# Add cluster centers and labels
centers = kmeans.cluster_centers_
aggregated_df['Experience Cluster'] = aggregated_df['Experience Cluster'].astype('category')

# Display cluster centers for interpretation
print("\nCluster Centers:")
print(centers)


C:\Users\HP EliteBook\AppData\Local\Temp\ipykernel_25360\928248976.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experience_metrics.fillna(experience_metrics.mean(), inplace=True)
c:\Anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")



Cluster Centers:
[[-1.19981031 -0.12203519  1.11966965]
 [ 0.62363849 -0.75641034 -0.50791393]
 [ 0.321499    1.18377276 -0.4040188 ]]


In [22]:
# Group by cluster to analyze the average values of each metric
cluster_analysis = aggregated_df.groupby('Experience Cluster')[['Avg_TCP_Retransmission', 'Avg_RTT', 'Avg_Throughput']].mean().reset_index()
print("\nAverage Metrics per Cluster:")
print(cluster_analysis)



Average Metrics per Cluster:
  Experience Cluster  Avg_TCP_Retransmission     Avg_RTT  Avg_Throughput
0                  0            3.717555e+06   62.930492    19449.935611
1                  1            1.974273e+07   38.641339     1438.989947
2                  2            1.711164e+07  112.665721     2584.453807


C:\Users\HP EliteBook\AppData\Local\Temp\ipykernel_25360\569490434.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cluster_analysis = aggregated_df.groupby('Experience Cluster')[['Avg_TCP_Retransmission', 'Avg_RTT', 'Avg_Throughput']].mean().reset_index()
